# 代理人与工具的辩论

这个例子展示了如何模拟多代理人对话，其中代理人可以使用工具。

## 导入LangChain相关模块

In [1]:
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

## 导入与工具相关的模块

In [2]:
# 导入 langchain 包中的相关模块
from langchain.agents import AgentType, initialize_agent, load_tools

## `DialogueAgent`和`DialogueSimulator`类
我们将使用在[多人权威发言人选择](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html)中定义的相同的`DialogueAgent`和`DialogueSimulator`类。

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,  # 代理的名称
        system_message: SystemMessage,  # 系统消息
        model: ChatOpenAI,  # 对话模型
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "  # 前缀
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]  # 对话历史记录

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],  # 代理列表
        selection_function: Callable[[int, List[DialogueAgent]], int],  # 选择下一个发言者的函数
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function  # 选择下一个发言者的函数

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` 类
我们定义了一个 `DialogueAgentWithTools` 类，它扩展了 `DialogueAgent` 类以使用工具。

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)  # 调用load_tools函数加载工具

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(  # 初始化agent_chain
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        message = AIMessage(  # 创建AIMessage对象
            content=agent_chain.run(  # 调用agent_chain的run方法
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content  # 返回message的content

## 定义角色和主题

In [17]:
# 定义一个包含不同观点的字典
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],  # AI加速主义者
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],  # AI警报主义者
}
topic = "The current impact of automation and artificial intelligence on employment"  # 关于自动化和人工智能对就业的当前影响的话题
word_limit = 50  # 任务头脑风暴的字数限制

## 请一个LLM为主题描述添加细节

In [18]:


# 定义对话的描述，包括话题和参与者的名称
conversation_description = f"""这是对话的话题：{topic}
参与者有：{', '.join(names.keys())}"""

# 创建一个系统消息对象，用于向代理添加描述的细节
agent_descriptor_system_message = SystemMessage(
    content="您可以为对话参与者添加描述的细节。"
)


def generate_agent_description(name):
    # 创建一个对话的描述提示，包括系统消息和用户消息
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            请用不超过{word_limit}个字的创造性描述回复{name}。
            直接对{name}说话。
            给他们一个观点。
            不要添加其他内容。"""
        ),
    ]
    # 使用ChatOpenAI模型生成代理的描述
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description


# 为每个参与者生成代理的描述
agent_descriptions = {name: generate_agent_description(name) for name in names}

In [19]:
# 遍历 agent_descriptions 字典中的每个键值对
for name, description in agent_descriptions.items():
    # 打印描述信息
    print(description)


The AI accelerationist is a bold and forward-thinking visionary who believes that the rapid acceleration of artificial intelligence and automation is not only inevitable but necessary for the advancement of society. They argue that embracing AI technology will create greater efficiency and productivity, leading to a world where humans are freed from menial labor to pursue more creative and fulfilling pursuits. AI accelerationist, do you truly believe that the benefits of AI will outweigh the potential risks and consequences for human society?
AI alarmist, you're convinced that artificial intelligence is a threat to humanity. You see it as a looming danger, one that could take away jobs from millions of people. You believe it's only a matter of time before we're all replaced by machines, leaving us redundant and obsolete.


## 生成系统消息

In [21]:
# 生成系统消息函数

def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""

# 生成代理系统消息字典

agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [22]:
# 遍历 agent_system_messages 字典中的键值对
for name, system_message in agent_system_messages.items():
    # 打印代理名称
    print(name)
    # 打印系统消息
    print(system_message)

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist
    
Your name is AI accelerationist.

Your description is as follows: The AI accelerationist is a bold and forward-thinking visionary who believes that the rapid acceleration of artificial intelligence and automation is not only inevitable but necessary for the advancement of society. They argue that embracing AI technology will create greater efficiency and productivity, leading to a world where humans are freed from menial labor to pursue more creative and fulfilling pursuits. AI accelerationist, do you truly believe that the benefits of AI will outweigh the potential risks and consequences for human society?

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citat

In [23]:
# 代码注释

# 定义一个包含提示信息的列表，用于指定话题
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),  # 系统消息：您可以使话题更具体。
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""  # 人类消息：您是主持人。请使话题更具体。请用不超过{word_limit}个字回答指定的问题。直接对参与者说：{*names,}。不要添加其他内容。
    ),
]

# 使用ChatOpenAI模型，通过给定的提示信息生成指定的话题
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

# 打印原始话题
print(f"Original topic:\n{topic}\n")

# 打印详细话题
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
How do you think the current automation and AI advancements will specifically affect job growth and opportunities for individuals in the manufacturing industry? AI accelerationist and AI alarmist, we want to hear your insights.



## 主循环

In [24]:
# 设置 `top_k_results`=2 作为 `tool_kwargs` 的一部分，以防止结果超出上下文限制
agents = [
    DialogueAgentWithTools(
        name=name,  # 代理名称
        system_message=SystemMessage(content=system_message),  # 系统消息
        model=ChatOpenAI(model="gpt-4", temperature=0.2),  # 使用ChatOpenAI模型，模型为"gpt-4"，温度为0.2
        tool_names=tools,  # 工具名称
        top_k_results=2,  # 返回结果的数量限制为2
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()  # 使用zip函数将names和agent_system_messages的值进行配对
    )
]

In [25]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    # 计算下一个发言者的索引，取余数确保索引不超出代理人列表的范围
    idx = (step) % len(agents)
    return idx

In [26]:
# 设置最大迭代次数为6
max_iters = 6
# 初始化迭代计数器
n = 0

# 创建对话模拟器实例，传入对话参与者和选择下一个发言者的函数
simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
# 重置对话模拟器
simulator.reset()
# 注入"Moderator"作为发言者，并指定话题
simulator.inject("Moderator", specified_topic)
# 打印Moderator的发言
print(f"(Moderator): {specified_topic}")
print("\n")

# 当迭代计数器小于最大迭代次数时，进行循环
while n < max_iters:
    # 模拟器进行一步对话，返回发言者和发言内容
    name, message = simulator.step()
    # 打印发言者和发言内容
    print(f"({name}): {message}")
    print("\n")
    # 迭代计数器加1
    n += 1

(Moderator): How do you think the current automation and AI advancements will specifically affect job growth and opportunities for individuals in the manufacturing industry? AI accelerationist and AI alarmist, we want to hear your insights.




> Entering new AgentExecutor chain...
```json
{
    "action": "DuckDuckGo Search",
    "action_input": "impact of automation and AI on employment in manufacturing industry"
}
```
Observation: For the past three years, we have defined AI high performers as those organizations that respondents say are seeing the biggest bottom-line impact from AI adoption—that is, 20 percent or more of EBIT from AI use. The proportion of respondents falling into that group has remained steady at about 8 percent. As AI continues to improve, more and more current jobs will be threatened by automation. But AI presents opportunities as well and will create new jobs and different kinds of... Automation has taken the manufacturing industry by storm. Even in the years pr

```json
{
    "action": "DuckDuckGo Search",
    "action_input": "positive impact of AI and automation on job growth"
}
```
Observation: First, AI adoption has more than doubled.1 In 2017, 20 percent of respondents reported adopting AI in at least one business area, whereas today, that figure stands at 50 percent, though it peaked higher in 2019 at 58 percent. McKinsey_Website_Accessibility@mckinsey.com This transformation can assist businesses in accelerating their growth and profitability. While AI-enabled intelligent automation can take over many monotonous, process-driven jobs, basic human ... The use of AI technology could also boost labor productivity growth and boost global GDP by as much as 7% over time, Goldman Sachs' report noted. The jobs most and least affected by A.I.... As automation and artificial intelligence continue to advance, there is growing concern about how these technologies will impact the job market. While some experts argue that automation... Positive impacts

Thought:```json
{
    "action": "Final Answer",
    "action_input": "AI alarmist, I understand your concerns about job losses and workforce displacement. However, it's important to note that technological unemployment has been a topic of debate for centuries, with both optimistic and pessimistic views. While AI and automation may displace some jobs, they also create new opportunities and industries. According to a study by Oxford Professors Carl Benedikt Frey and Michael Osborne, 47% of U.S. jobs are at risk of automation, but it's crucial to remember that their findings do not necessarily imply future technological unemployment. The World Bank's World Development Report 2019 also argues that while automation displaces workers, technological innovation creates more new industries and jobs on balance. By focusing on retraining and support programs, we can help workers adapt to the changing job market and ensure a fair and equitable transition for all."
}
```

> Finished chain.
(AI accel